In [1]:
#### Importing all necessary python Modules ######

import csv,operator,re,itertools
import pandas as pd

from string import punctuation
from collections import Counter

from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

In [6]:
dict_smileys = {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

dict_contractions = {
            "ain't":"is not",
            "amn't":"am not",
            "aren't":"are not",
            "can't":"can not",
            "'cause":"because",
            "couldn't":"could not",
            "couldn't've":"could not have",
            "could've":"could have",
            "daren't":"dare not",
            "daresn't":"dare not",
            "dasn't":"dare not",
            "didn't":"did not",
            "doesn't":"does not",
            "don't":"do not",
            "e'er":"ever",
            "em":"them",
            "everyone's":"everyone is",
            "finna":"fixing to",
            "gimme":"give me",
            "gonna":"going to",
            "gon't":"go not",
            "gotta":"got to",
            "hadn't":"had not",
            "hasn't":"has not",
            "haven't":"have not",
            "he'd":"he would",
            "he'll":"he will",
            "he's":"he is",
            "he've":"he have",
            "how'd":"how would",
            "how'll":"how will",
            "how're":"how are",
            "how's":"how is",
            "i'd":"i would",
            "i'll":"i will",
            "i'm":"i am",
            "im":"i am",
            "i'm'a":"i am about to",
            "i'm'o":"i am going to",
            "isn't":"is not",
            "it'd":"it would",
            "it'll":"it will",
            "it's":"it is",
            "i've":"i have",
            "kinda":"kind of",
            "let's":"let us",
            "mayn't":"may not",
            "may've":"may have",
            "mightn't":"might not",
            "might've":"might have",
            "mustn't":"must not",
            "mustn't've":"must not have",
            "must've":"must have",
            "needn't":"need not",
            "ne'er":"never",
            "o'":"of",
            "o'er":"over",
            "ol'":"old",
            "oughtn't":"ought not",
            "shalln't":"shall not",
            "shan't":"shall not",
            "she'd":"she would",
            "she'll":"she will",
            "she's":"she is",
            "shouldn't":"should not",
            "shouldn't've":"should not have",
            "should've":"should have",
            "somebody's":"somebody is",
            "someone's":"someone is",
            "something's":"something is",
            "that'd":"that would",
            "that'll":"that will",
            "that're":"that are",
            "that's":"that is",
            "there'd":"there would",
            "there'll":"there will",
            "there're":"there are",
            "there's":"there is",
            "these're":"these are",
            "they'd":"they would",
            "they'll":"they will",
            "they're":"they are",
            "they've":"they have",
            "this's":"this is",
            "those're":"those are",
            "'tis":"it is",
            "'twas":"it was",
            "wanna":"want to",
            "wasn't":"was not",
            "we'd":"we would",
            "we'd've":"we would have",
            "we'll":"we will",
            "we're":"we are",
            "weren't":"were not",
            "we've":"we have",
            "what'd":"what did",
            "what'll":"what will",
            "what're":"what are",
            "what's":"what is",
            "what've":"what have",
            "when's":"when is",
            "where'd":"where did",
            "where're":"where are",
            "where's":"where is",
            "where've":"where have",
            "which's":"which is",
            "who'd":"who would",
            "who'd've":"who would have",
            "who'll":"who will",
            "who're":"who are",
            "who's":"who is",
            "who've":"who have",
            "why'd":"why did",
            "why're":"why are",
            "why's":"why is",
            "won't":"will not",
            "wouldn't":"would not",
            "would've":"would have",
            "y'all":"you all",
            "you'd":"you would",
            "you'll":"you will",
            "you're":"you are",
            "you've":"you have",
            "Whatcha":"What are you",
            "luv":"love",
            "sux":"sucks"
            }


In [28]:
def clean_data(text):    
    
    #Escaping HTML characters

    text = text.lower()

    text = text.replace("’","'")

    tokens = text.split()
    
    tokens = [dict_contractions[word] if word in dict_contractions else word for word in tokens]

    tokens = [dict_smileys[word] if word in dict_smileys else word for word in tokens]

    text = ' '.join(tokens)

    text = re.sub(r"http\S+", "", text)
   
    #Special case not handled previously.
    text = text.replace('\x92',"'")

    #removing retweet tag
    text = text.replace('RT',"")
    
    #Removal of hastags
    text = re.sub(r"#[A-Za-z0-9]+", "", text)

    #Removal of handle
    text = re.sub(r"@[A-Za-z0-9_]+", "", text,5)
    
    #Removal of address
    text = re.sub("(\w+:\/\/\S+)", "", text)

    #Removal of Punctuation
    text = re.sub("[\.\,\!\?\:\;\-\=]", " ", text)

    text = re.sub("[^A-Za-z ]", "", text)

    tokens = text.split()
    
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]

    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]

    text = ' '.join(tokens)

    #print(text)
    
    return text

from random import randint

df = pd.read_csv('dataset/train_data.csv',encoding='latin-1')

words = df['text']

i = randint(0,10000)

print(words[i],clean_data(words[i]),sep='\n')

 the AC is on!!
ac


In [29]:
df = pd.read_csv('dataset/train_data.csv',encoding='latin-1')

print(df)

words = df['text']

y = df['Sentiment']

       ItemID  Sentiment                                               text
0           1          0                       is so sad for my APL frie...
1           2          0                     I missed the New Moon trail...
2           3          1                            omg its already 7:30 :O
3           4          0            .. Omgaga. Im sooo  im gunna CRy. I'...
4           5          0           i think mi bf is cheating on me!!!   ...
...       ...        ...                                                ...
99984   99996          0  @Cupcake  seems like a repeating problem   hop...
99985   99997          1  @cupcake__ arrrr we both replied to each other...
99986   99998          0                     @CuPcAkE_2120 ya i thought so 
99987   99999          1  @Cupcake_Dollie Yes. Yes. I'm glad you had mor...
99988  100000          1                    @cupcake_kayla haha yes you do 

[99989 rows x 3 columns]


In [31]:
fp = open('dataset/clean_data.csv','w')

fp.write('text,Sentiment')

for i in range(0,len(words)):

    w = clean_data(words[i])

    if(len(w)<20):

        continue

    fp.write('\n'+w+','+str(y[i]))
    
df = pd.read_csv('dataset/clean_data.csv')

print(df)

                                                    text  Sentiment
0                                missed new moon trailer          0
1      omgaga sooo gunna cry dentist since suposed ge...          0
2                                think mi bf cheating tt          0
3                       juuuuuuuuuuuuuuuuussssst chillin          1
4                         sunny work tomorrow tv tonight          0
...                                                  ...        ...
74857                            prada linea rossa shoes          1
74858                   hey love delores park sf anymore          0
74859  well try carolyn although would lying said fin...          1
74860  ok thanks let know think going go play box men...          1
74861  brittish parliament challenges clearly puppete...          1

[74862 rows x 2 columns]
